In [1]:
"""
To make the tables and methods for the platinum blender tool that is designed to: 
1) Import the main neuron and all of its connected parts
2) Import the synapses associated with all of those pieces
- label the synapses as those in synapse exclude or not
3) Captur the label labels and write them to the Proofreading table
4) Translate the labels to the synapses

"""

'\nTo make the tables and methods for the platinum blender tool that is designed to: \n1) Import the main neuron and all of its connected parts\n2) Import the synapses associated with all of those pieces\n- label the synapses as those in synapse exclude or not\n3) Captur the label labels and write them to the Proofreading table\n4) Translate the labels to the synapses\n\n'

In [7]:
import datajoint as dj
import numpy as np

In [8]:
x = np.array([[1,2,3],[3,4,5]])
y = x*[4, 4, 40]
y

array([[  4,   8, 120],
       [ 12,  16, 200]])

In [9]:
pinky = dj.create_virtual_module("pinky","microns_pinky")


/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:215: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
pinky.ProofreadLabelOrphan()

# How to interface with the tables that has platinum data nad connectivity

In [10]:
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

In [11]:
m65.WholeAutoAnnotationsLabelClusters3()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,decimation_ratio percentage that the faces are decimated,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices label data for the vertices,triangles label data for the faces,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey
96631955273149705,0,1.00,computer_Auto,bcelii,2019-11-07 19:04:40,=BLOB=,=BLOB=,=BLOB=,complete
107816118160698192,0,1.00,computer_Auto,bcelii,2019-11-07 19:04:47,=BLOB=,=BLOB=,=BLOB=,complete


In [ ]:
m65.FromNeuromancer()

In [ ]:
m65.Synapse()

In [ ]:
m65.FromNeuromancer.Subsegment()
96631955273149705

# Concatenating the Mesh: 

In [7]:
m65.FromNeuromancer()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,neuron_name Name of the neuron as it appears in the Neuromancer-Ingest website.,n_vertices,n_faces,vertices,faces
96631955273149705,0,RONNY,1659719,3161800,=BLOB=,=BLOB=
107816118160698192,0,CHARMAINE,937779,1791632,=BLOB=,=BLOB=


In [5]:
#have to adjust the coordinates by * [4, 4, 40])
# practice on getting all of the synapses associated with a main segment and 
neurons_data = m65.FromNeuromancer.fetch(as_dict=True)

In [8]:
for n in neurons_data:
    print("---Main Neuron ---")
    print(n["segment_id"])
    print(n["n_vertices"])
    print(n["vertices"])
    print(n["faces"])
    
    #get all of the segments and their data
    lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
    subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)
    
    subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])
    
    subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
    subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
    subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]
    
    for sub in subsegments:
        pass
#         print(f"------sub_segment = {sub['subsegment_id']}-------")
#         print(sub["n_vertices"])
#         print(sub["vertices"])
#         print(sub["faces"])

---Main Neuron ---
96631955273149705
1659719
[[ 848887.5     776986.25   1032329.625 ]
 [ 848993.      776912.875  1032339.8125]
 [ 848950.625   777083.0625 1032342.    ]
 ...
 [1083392.      758736.     1069720.    ]
 [1083392.      758160.     1070520.    ]
 [1083392.      758144.     1070540.    ]]
[[      0       2       1]
 [      0       1       3]
 [      4       6       5]
 ...
 [1656311 1655595 1657388]
 [1657331 1655945 1657332]
 [1658685 1657387 1658709]]
---Main Neuron ---
107816118160698192
937779
[[1219061.75     459752.59375  865277.5    ]
 [1218984.5      459845.03125  865190.25   ]
 [1219070.       459819.375    865243.25   ]
 ...
 [1360432.       734720.       787360.     ]
 [1360528.       734784.       787360.     ]
 [1360640.       734832.       787360.     ]]
[[     0      2      1]
 [     0      1      3]
 [     0      3      4]
 ...
 [937729 937765 937778]
 [937729 937778 937758]
 [937712 937711 937767]]


In [ ]:
import numpy as np
print((len(n["vertices"]),len(sub["vertices"]),len(n["vertices"])+len(sub["vertices"])))
new_verts = np.vstack([n["vertices"],sub["vertices"]])
print(len(new_verts))
new_verts

In [9]:
#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

In [ ]:
new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
                   main_mesh_faces=n["faces"],
                   sub_mesh_vertices = subsegments_vertices,
                   sub_mesh_faces=subsegments_faces)

In [ ]:
# test the mesh concatenator
import trimesh
original_main = trimesh.Trimesh(vertices=n["vertices"],faces=n["faces"])

new_mesh = trimesh.Trimesh(vertices=new_mesh_vertices,faces=new_mesh_faces)
new_mesh.show()

In [ ]:
original_main = trimesh.Trimesh(vertices=n["vertices"],faces=n["faces"])
#original_main.show()

In [ ]:
new_mesh.export("extended_mesh.off")

# Dealing with Synapses

In [ ]:
"""
Find the synapses associated with the neuron main part and all it's pieces
Pseudocode: 
1) Get all of the main segment and subsegments ids
2) Restrict the synapse table to all where those are the presyn or postsyn
3) Pull down the synapse id, and the locations
4) Go to the synapse exclude and restruct by the synape ids
5) Pull down any id's that are there

End result: Have list of ids and their coordinate plus list of which ones are in synapse Exclude
"""

In [ ]:
#making the table for the Exclusion Criteria
@schema
class ExclusionCriteria(dj.Manual):
    definition="""
    criteria_id          : int unsigned                 # key by which to lookup the exclusion criteria description
    ---
    criteria_desc        : varchar(64)                  # criteria by which this segment was excluded (preferred to add name of excluder as well)
    """
    
#ExclusionCriteria.insert1(dict(criteria_id=0,criteria_desc="Blender Error Labeling"))

In [18]:
m65.ExclusionCriteria()

criteria_id key by which to lookup the exclusion criteria description,criteria_desc criteria by which this segment was excluded (preferred to add name of excluder as well)
0,Blender Error Labeling
1,Celii test


In [23]:
m65.SynapseExclude.drop()

`microns_minnie65_01`.`synapse_exclude` (0 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.


In [24]:
#making the table for the SynapseExclude
@schema
class SynapseExclude(dj.Manual):
    definition="""
    -> m65.Synapse
    version              : tinyint unsigned             # version of proofreading for the neuron
    -> m65.ExclusionCriteria
    ---
    -> m65.Segment.proj(presyn="segment_id")
    -> m65.Segment.proj(postsyn="segment_id")
    """
SynapseExclude()
#ExclusionCriteria.insert1(dict(criteria_id=0,criteria_desc="Blender Error Labeling"))

synapse_id synapse index within the segmentation,version version of proofreading for the neuron,criteria_id key by which to lookup the exclusion criteria description,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation


In [ ]:
#### just filling synapse exclude and exclusion criteria with some dummy values for testing
# m65.ExclusionCriteria.insert1([1,"Celii test"])
m65.ExclusionCriteria()

example_dict_1 = dict(synapse_id=317393639,version=1,criteria_id=0,presyn=92622516933251392,postsyn=96631955273149705)
example_dict_2 = dict(synapse_id=317448087,version=1,criteria_id=0,presyn=91777679686517288,postsyn=96631955273149705)
m65.SynapseExclude.insert([example_dict_1,example_dict_2])

In [8]:
main_neuron_id = 96631955273149705
#1) Get all the subsegments associated with main neuron
main_neuron_dj = dj.U("seg_id") & (m65.FromNeuromancer.Subsegment.proj(seg_id="segment_id") & "seg_id=" + str(main_neuron_id))
subseg_dj= dj.U("seg_id") & (m65.FromNeuromancer.Subsegment.proj(seg_id="subsegment_id") & "segment_id=" + str(main_neuron_id))
total_neurons = (main_neuron_dj + subseg_dj)

#Restrict Synapse Table by that list
presyn_list = m65.Synapse & total_neurons.proj(presyn="seg_id")
postsyn_list = m65.Synapse & total_neurons.proj(postsyn="seg_id")
total_synapse_ids =postsyn_list.proj() + presyn_list.proj()
synapse_data = m65.Synapse & total_synapse_ids
synapse_data_fetched = synapse_data.fetch(as_dict=True)

synapses_already_excluded = (synapse_data & m65.SynapseExclude.proj()).fetch("synapse_id")
synapses_already_excluded

array([317393639, 317448087])

In [14]:
len(synapse_data_fetched)/2

824.0

In [18]:
half_synapse_data[0]

OrderedDict([('synapse_id', 317393639),
             ('presyn', 92622516933251392),
             ('postsyn', 96631955273149705),
             ('centroid_x', 210902),
             ('centroid_y', 213514),
             ('centroid_z', 22807),
             ('size', 704)])

In [24]:
half_synapse_data = synapse_data_fetched[:int(len(synapse_data_fetched)/2)]
synapse_to_exclude = [dict(synapse_id=k["synapse_id"],version=0,criteria_id=1,presyn=k["presyn"],postsyn=k["postsyn"]) for k in half_synapse_data]

In [26]:
m65.SynapseExclude.insert(synapse_to_exclude)

In [27]:
m65.SynapseExclude()

synapse_id synapse index within the segmentation,version version of proofreading for the neuron,criteria_id key by which to lookup the exclusion criteria description,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation
317393639,0,1,92622516933251392,96631955273149705
317448087,0,1,91777679686517288,96631955273149705
318086051,0,1,92481160969984222,96631955273149705
318748884,0,1,96000491598978753,96631955273149705
318750436,0,1,96140404386728637,96631955273149705
318750734,0,1,96070035642567029,96631955273149705
319360255,0,1,96492385546494957,96631955273149705
320289505,0,1,95576286202087391,96631955273149705
320291110,0,1,96772279975167152,96631955273149705
320291360,0,1,96349655193160583,96631955273149705


In [ ]:
m65.FromNeuromancer.Subsegment()

In [ ]:
m65.FromNeuromancer.Subsegment() & "subsegment_id=93608573174803949 OR n_vertices=32"

In [ ]:
# How to get list of everything
#main_neuron_dj = dj.U("seg_id") & (m65.FromNeuromancer.Subsegment.proj(seg_id="segment_id") & "seg_id=" + str(main_neuron_id))
#subseg_dj= dj.U("seg_id") & (m65.FromNeuromancer.Subsegment.proj(seg_id="subsegment_id") & "segment_id=" + str(main_neuron_id))
#main_neuron_dj + subseg_dj

In [ ]:
# ----------------How to create a new object and color it in blender ------------ #

# mymesh = bpy.data.meshes.new("neuron-"+ID)
# mymesh.from_pydata(verts, [], faces)
# mymesh.update(calc_edges=True)
# mymesh.calc_normals()
# #print(objects_Matching_filename)
# object = bpy.data.objects.new("neuron-"+ID, mymesh)
# #object.location = bpy.context.scene.cursor_location
# object.location = Vector((0,0,0))
# bpy.context.scene.objects.link(object)


# object.lock_location[0] = True
# object.lock_location[1] = True
# object.lock_location[2] = True
# object.lock_scale[0] = True
# object.lock_scale[1] = True
# object.lock_scale[2] = True

# #rotate the z direction by 90 degrees so point correct way

# #object.rotation_euler[2] = 1.5708


# object.rotation_euler[0] = 4.7124
# object.rotation_euler[1] = 0
# object.rotation_euler[2] = 0




# object.lock_rotation[0] = True
# object.lock_rotation[1] = True
# object.lock_rotation[2] = True

# #run the setup color command
# #bpy.ops.object.select_all(action='TOGGLE')
# create_local_colors(object)


# #then how to label the synapses: 
# #print(len(active_edges_raw))
        
# faces_raw = object.data.polygons
# for i,k in enumerate(faces_raw):
#     k.material_index = int(triangles_labels[i])





In [ ]:
"""
Brainstorming how to make the synapse sphere: Best strategy is just to make the boxes

Prebuilt function
1) Box maker: 
- input: takes in box center and the side length
- returns: vertices and faces for a box mesh with that center and side length

Pseudocode: 
1)Iterate through all synapses:
- send the center to the box maker function and save in list
- if in the synapse exclude table then save the face numbers in a list 
(accounting for where all the face numbers will be once concatenated)
2) Use the mesh concatenation to put all the faces together
3) Send to blender as the mesh and use the face number of the synapse exclude to color brown

"""

In [ ]:
box_mesh.export("box_synapses.off")
print("ok")

In [ ]:
# total synapse function

"""
Practice going and getting all synapse for that main mesh and all subsegments (with their stats)
"""
import numpy as np

def get_synapses_with_exclude_list(main_neuron_id):
    #main_neuron_id=96631955273149705
    """
    Purpose: getting all synapse for that main mesh and all subsegments (with their stats)

    Pseudocode: 
    1) Get all the subsegments associated with main neuron
    2) Restrict Synapse Table by that list
    3) Pull down the synapse id's, coordinates
    4) Use synapse mini table to restrict the synapse exclude and pull down all id's in there
    5) return the synapse ids with their coordinates in dictionary form and a list of those syanses in synapse exclud
    
    How to use the function:
    synapse_data_fetched,synapses_already_excluded = get_synapses_with_exclude_list(96631955273149705)
    print("Number of synapses founded = " + str(len(synapse_data_fetched)))
    print("synapses_already_excluded = " + str(synapses_already_excluded))
    """

    #1) Get all the subsegments associated with main neuron
    main_neuron_dj = dj.U("seg_id") & (m65.FromNeuromancer.Subsegment.proj(seg_id="segment_id") & "seg_id=" + str(main_neuron_id))
    subseg_dj= dj.U("seg_id") & (m65.FromNeuromancer.Subsegment.proj(seg_id="subsegment_id") & "segment_id=" + str(main_neuron_id))
    total_neurons = (main_neuron_dj + subseg_dj)

    #Restrict Synapse Table by that list
    presyn_list = m65.Synapse & total_neurons.proj(presyn="seg_id")
    postsyn_list = m65.Synapse & total_neurons.proj(postsyn="seg_id")
    total_synapse_ids =postsyn_list.proj() + presyn_list.proj()
    synapse_data = m65.Synapse & total_synapse_ids
    synapse_data_fetched = synapse_data.fetch(as_dict=True)

    synapses_already_excluded = (synapse_data & m65.SynapseExclude.proj()).fetch("synapse_id")
    synapses_already_excluded
    
    return synapse_data_fetched,synapses_already_excluded

# box maker function
def create_synapse_box(center,side_length):
    """
    Purpose: Will create the mesh variables of vertices and faces
    for a box with a specified center and side length
    
    Parameters:
    center (tuple/np.array) : list of the x,y,z coordinates of the box
    side_length(float)      : the length that the sides need to be of box
    
    Returns: 
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for the box
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for the box
    
    How to use function: 
    create_synapse_box((2,2,2),2.5)
    """
    
    unit_box_vertices = np.array([
        [0,0,0],
        [1,0,0],
        [1,1,0],
        [0,1,0],
        [0,0,1],
        [1,0,1],
        [1,1,1],
        [0,1,1],
    ])

    unit_box_faces = np.array([
        [0,4,5],
        [0,5,1],
        [1,5,6],
        [1,6,2],
        [2,6,7],
        [2,7,3],
        [3,7,4],
        [3,4,0],
        [5,4,7],
        [6,5,7],
        [1,2,0],
        [3,0,2]

        ])

    #Now need to shift it down to center
    unit_box_vertices = unit_box_vertices - 0.5
    
    unit_box_vertices = unit_box_vertices*side_length
    unit_box_vertices = unit_box_vertices + np.array(center)
    
    return unit_box_vertices,unit_box_faces




def get_synapse_mesh_and_errored_faces(primary_key,return_faces_to_color=True):
    synapse_data_fetched,synapses_already_excluded = get_synapses_with_exclude_list(primary_key["segment_id"])
    
    # 1)Iterate through all synapses:
    # - send the center to the box maker function and save in list
    # - if in the synapse exclude table then save the face numbers in a list 

    face_count = 0
    faces_to_color = []
    synapse_vertices = []
    synapse_faces = []

    for i,syn in enumerate(synapse_data_fetched):
    #     if i < 5:
    #         print(syn["synapse_id"])
        center = (syn["centroid_x"],syn["centroid_y"],syn["centroid_z"])
        center = center*np.array([4, 4, 40])
        side_length = 300
        unit_box_vertices,unit_box_faces = create_synapse_box(center,side_length)
        #add them to the list
        synapse_vertices.append(unit_box_vertices)
        synapse_faces.append(unit_box_faces)

        #check if synapse in already excluded list
        syn_id = syn["synapse_id"]
        if syn_id in synapses_already_excluded:
            #added the faces to color list
            #print(i)
            faces_to_exclude = list(range(face_count,face_count + len(unit_box_faces)))
            #print(faces_to_exclude)
            faces_to_color = faces_to_color + faces_to_exclude

        face_count += len(unit_box_faces)

    print(faces_to_color)
    # 2) Use the mesh concatenation to put all the faces together
    box_mesh_verts, box_mesh_faces = add_mesh_piece(synapse_vertices[0],synapse_faces[0],synapse_vertices[1:],synapse_faces[1:])
    
    return box_mesh_verts,box_mesh_faces,faces_to_color

current_decimation = 1.0
current_version = 0
ID = 96631955273149705
primary_key = dict(version=current_version,decimation_ratio=current_decimation,segment_id=int(ID)) 
box_mesh_verts,box_mesh_faces,faces_to_color = get_synapse_mesh_and_errored_faces(primary_key)


box_mesh = trimesh.Trimesh(vertices=box_mesh_verts,faces=box_mesh_faces)
box_mesh.show()

# Set up the Proofread Table that autopopulates from the large segments table

In [ ]:
labels_Table = pinky.WholeAutoAnnotationsLabelClusters3()
proof_Table = pinky.ProofreadLabel()

In [ ]:
proof_Table.describe()
print("")

In [ ]:
proof_Table.describe()

In [ ]:
labels_Table

In [ ]:
pinky.Decimation35.describe()

In [ ]:
(pinky.WholeAutoAnnotationsLabelClusters3 & "segment_id=648518346341371119").fetch1()

In [ ]:
np.array([])

In [ ]:
m65.FromNeuromancer()

In [ ]:
pinky.ProofreadLabel.describe()

In [11]:
m65.WholeAutoAnnotationsLabelClusters3.delete()

About to delete:
`microns_minnie65_01`.`__whole_auto_annotations_label_clusters3`: 1 items


Proceed? [yes, No]:  yes


Committed.


In [12]:
m65.FromNeuromancer()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,neuron_name Name of the neuron as it appears in the Neuromancer-Ingest website.,n_vertices,n_faces,vertices,faces
96631955273149705,0,RONNY,1659719,3161800,=BLOB=,=BLOB=
106626583548494129,0,WILSON,2938478,5601980,=BLOB=,=BLOB=
106626583548494129,1,WILSON,2938478,5601980,=BLOB=,=BLOB=
107816118160698192,0,CHARMAINE,937779,1791632,=BLOB=,=BLOB=


In [21]:
verts,tri_labels = (m65.WholeAutoAnnotationsLabelClusters3 & "segment_id=106626583548494129" & "version=1").fetch1("vertices","triangles")
len(verts),len(tri_labels)

(3633537, 6951402)

In [14]:
import datetime

#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

@schema
class WholeAutoAnnotationsLabelClusters3(dj.Computed):
    definition="""
    -> m65.FromNeuromancer
    decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
    """
    
    def make(self,key):
        """
        Pseudocode: 
        1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
        3) Create blank labels for the size of the vertices, triangles and edges and make parameters:
        a. decimation = 1
        author_original = computer_Auto
        author_proofreader = bcelii
        date_time = current time
        status = complete
        4) Insert the key into the table
        """
        
        #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
        n = (m65.FromNeuromancer & key).fetch1()
        
        print("---Main Neuron ---")
        print(n["segment_id"])
        print(n["n_vertices"])
        print(n["vertices"])
        print(n["faces"])

        #get all of the segments and their data
        lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
        subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)

        subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

        subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
        subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
        subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]
        
        
        new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
                   main_mesh_faces=n["faces"],
                   sub_mesh_vertices = subsegments_vertices,
                   sub_mesh_faces=subsegments_faces)
        
        #3) Create blank labels for the size of the vertices, triangles and edges
        
        insert_key = dict(key)
        insert_key["vertices"] = np.zeros(len(new_mesh_vertices))
        insert_key["edges"] = np.array([])
        insert_key["triangles"] = np.zeros(len(new_mesh_faces))
        insert_key["decimation_ratio"] = 1.0
        insert_key["author_original"] = "computer_Auto"
        insert_key["author_proofreader"] = "bcelii"
        insert_key["status"] = "complete"
        insert_key["date_time"] = date_time = str(datetime.datetime.now())[0:19]
        
        #4) Insert the key into the table
        self.insert1(insert_key,skip_duplicates=True)
        

In [17]:
m65.WholeAutoAnnotationsLabelClusters3()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,decimation_ratio percentage that the faces are decimated,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices label data for the vertices,triangles label data for the faces,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey
96631955273149705,0,1.00,computer_Auto,bcelii,2019-11-07 19:04:40,=BLOB=,=BLOB=,=BLOB=,complete
106626583548494129,0,1.00,computer_Auto,bcelii,2019-11-16 03:25:14,=BLOB=,=BLOB=,=BLOB=,complete
106626583548494129,1,1.00,computer_Auto,bcelii,2019-11-16 03:25:31,=BLOB=,=BLOB=,=BLOB=,complete
107816118160698192,0,1.00,computer_Auto,bcelii,2019-11-07 19:04:47,=BLOB=,=BLOB=,=BLOB=,complete


In [16]:
#(schema.jobs & "table_name='__whole_auto_annotations_label_clusters3'")#.delete()
dj.config["enable_python_native_blobs"] = True

import time
start_time = time.time()
WholeAutoAnnotationsLabelClusters3.populate(reserve_jobs=True)
print(f"Total time for WholeAutoAnnotationsLabelClusters3 populate = {time.time() - start_time}")

---Main Neuron ---
106626583548494129
2938478
[[1079400.5      469942.59375  689338.4375 ]
 [1079360.       469888.       689340.     ]
 [1079287.75     469859.25     689376.25   ]
 ...
 [1436672.       926848.       751820.     ]
 [1436672.       926464.       751620.     ]
 [1436672.       926736.       751560.     ]]
[[      0       2       1]
 [      0       1       3]
 [      0       3       4]
 ...
 [2938327 2938477 2938326]
 [2938230 2938477 2938327]
 [2938230 2938468 2938477]]
Added 69 subsegements 
  --> final mesh: 3595489 vertices and 6878350 faces
---Main Neuron ---
106626583548494129
2938478
[[1079400.5      469942.59375  689338.4375 ]
 [1079360.       469888.       689340.     ]
 [1079287.75     469859.25     689376.25   ]
 ...
 [1436672.       926848.       751820.     ]
 [1436672.       926464.       751620.     ]
 [1436672.       926736.       751560.     ]]
[[      0       2       1]
 [      0       1       3]
 [      0       3       4]
 ...
 [2938327 2938477 2938326]

In [15]:
m65.ProofreadLabel.delete()

About to delete:
`microns_minnie65_01`.`proofread_label`: 2 items


Proceed? [yes, No]:  yes


Committed.


In [26]:
ProofreadLabel.delete()

About to delete:
`microns_minnie65_01`.`proofread_label`: 2 items


Proceed? [yes, No]:  yes


Committed.


In [16]:
@schema
class ProofreadLabel(dj.Manual):
    definition="""
    -> m65.FromNeuromancer
    decimation_ratio     : decimal(3,2)                 # percentage that the faces are decimated
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
    """

In [ ]:
#create the authors table
pinky.Authors()

In [ ]:
pinky.Authors.describe()
print("hello")

@schema
class Authors(dj.Manual):
    definition="""
    # maps numeric labels to descriptive labels
    username             : varchar(20)                  # username the person pcisk
    ---
    real_name            : varchar(40)                  # the real name of that corresponds to the username
    """
    
Authors.insert([dict(username="celiib",real_name="Brendan Celii"),dict(username="spapadop",real_name="stelios papadopoulos")])

In [ ]:
# creat the new LabelKey
total_Label_data = pinky.LabelKey.fetch()

@schema
class LabelKey(dj.Manual):
    definition="""
    # maps numeric labels to descriptive and color labels
    numeric              : tinyint unsigned             # numeric label of the compartment
    ---
    description          : varchar(16)                  # descriptive name of the label
    color                : varchar(16)                  # representative color of the compartment
    blender_colors       : tinyblob                     # color RGB values to be used in blender
    """
    
LabelKey.insert(total_Label_data)

In [ ]:
#need a table where can store the final meshes


In [ ]:
#thing have to change for blender
"""
segmentation --> version
number 3 of segmentation to 0 (or current version)

Have function for going and getting the mesh and mesh number instead of Pymeshfix table
"""

In [ ]:
def fetch_stitched_mesh(key,return_list=True):

    #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
    n = (m65.FromNeuromancer & key).fetch1()

    print("---Main Neuron ---")
    print(n["segment_id"])
    print(n["n_vertices"])
    print(n["vertices"])
    print(n["faces"])

    #get all of the segments and their data
    lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
    subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)

    subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

    subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
    subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
    subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]


    new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
       main_mesh_faces=n["faces"],
       sub_mesh_vertices = subsegments_vertices,
       sub_mesh_faces=subsegments_faces)
    if return_list == False:
        return new_mesh_vertices.astype(dtype=np.int32),new_mesh_faces.astype(dtype=np.int32)
    else:
        return new_mesh_vertices.astype(dtype=np.int32).tolist(),new_mesh_faces.astype(dtype=np.int32).tolist()

def fetch_stitched_mesh_statistics(key):

    #1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
    main_n_vertices,main_n_faces = (m65.FromNeuromancer & key).fetch1("n_vertices","n_faces")

    #get all of the segments and their data
    lookup_key = key
    subsegments_verts,subsegments_faces = (m65.FromNeuromancer.Subsegment & lookup_key).fetch("n_vertices","n_faces")
    
    
    
    return np.sum(subsegments_verts) + main_n_vertices,np.sum(subsegments_faces) + main_n_faces

In [ ]:
lookup_key = dict(segment_id=96631955273149705,version=0)
fetch_stitched_mesh(lookup_key)
print(fetch_stitched_mesh_statistics(lookup_key))

In [ ]:
m65.FromNeuromancer()

In [ ]:
m65.FromNeuromancer.Subsegment()

In [ ]:
m65.ProofreadLabel()